# Data Wrangling Challenge
### Pull and manipulate the API data

The point of this exercise is to try data enrichment with data from external APIs. We are going to take data about car crashes in Monroe County, Indiana from 2003 to 2015 and try to figure out the weather during the accident and how many bars there are in the area. We will work with two different APIs during this challenge:

- Foursquare API
- World Weather Online API

We will try to find correlations between the severity of crash and weather/number of bars in the area. To indicate the severity of a crash, we will use column `Injury Type`.

## Data

The data for this exercise can be found [here](https://drive.google.com/file/d/1_KF9oIJV8cB8i3ngA4JPOLWIE_ETE6CJ/view?usp=sharing).

Just run the cells below to get your data ready. Little help from us.


In [1]:
import pandas as pd
import os

In [2]:
data = pd.read_csv("monroe-county-crash-data2003-to-2015.csv", encoding="unicode_escape")
# ========================
# preparing data
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data = data[data['ll'] != '0.0,0.0']
print(data.shape)
data.head()

(49005, 13)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,ll
0,902363382,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015,1,6,Weekday,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015,1,6,Weekend,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015,1,7,Weekend,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015,1,7,Weekend,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


# Foursquare API

Foursquare API documentation is [here](https://developer.foursquare.com/)

1. Start a foursquare application and get your keys.
2. For each crash, create the function **get_venues** that will pull bars in the radius of 5km around the crash

#### example
`get_venues('48.146394, 17.107969')`

3. Find a relationship (if there is any) between number of bars in the area and severity of the crash.

HINTs: 
- check out python package "foursquare" (no need to send HTTP requests directly with library `requests`)
- **categoryId** for bars and nightlife needs to be found in the [foursquare API documentation](https://developer.foursquare.com/docs/api-reference/venues/search/)

In [4]:
#set the keys
# foursquare_id = os.environ[""]
# foursquare_secret = os.environ[""]
foursquare_api = os.environ["FOURSQUARE_API_KEY"]

In [93]:
import requests

url = "https://api.foursquare.com/v3/places/search?"

headers = {
    "Accept": "application/json",
    "Authorization": foursquare_api
}

# latlong = "ll=39.15920668%2C-86.52587356"

radius = "radius=5000"

categoryId = "categories=13003"

# response = requests.request("GET", url+latlong+'&'+radius+'&'+categoryId, headers=headers)

# print(response.text)

In [ ]:
# def get_venues(ll):
#     response = requests.request("GET", url+latlong+'&'+radius+'&'+categoryId, headers=headers)

In [94]:
# iterate through each row in the dataframe and do an api call that will return the number of bars for each ll

# create a dictionary to store the latitude and longitude of each crash and the number of bars near that crash

bar_crash_dict = {'latlong':[],
                  'n_bars':[]}
bar_crash_dict

{'latlong': [], 'n_bars': []}

In [97]:
# get lat and long of each crash

for row in list(data.iterrows()):  # iterate through each row in the in the dataframe
    latlong = row[1]['ll']  # get the latitude and longitude of each crash
    bar_crash_dict['latlong'].append(latlong)  # add latlong to dictionary
# bar_crash_dict['latlong'][:5]

In [99]:
# get number of venues near the crash

for crash_loc in bar_crash_dict['latlong'][:5]: # iterate through each latlong in the bar_crash_dict
    response = requests.request("GET", url+crash_loc.replace(',','%2C')+'&'+radius+'&'+categoryId, headers=headers) # for each crash location, make a call to the api to return the bars nearby
    bars = response.json() # store the response in a json object
    bars_df = pd.json_normalize(bars, record_path ='results') # convert the json object into a dataframe
    bar_count = len(bars_df['name']) # count the number of bars in the dataframe
    bar_crash_dict['n_bars'].append(bar_count) # append the count of bars to the dictionary
    # get_venues(latlong)
# print(bar_crash_dict['latlong'][:5])

In [104]:
# get lat and long of each crash

latlong_list = []

for row in list(data.iterrows()):  # iterate through each row in the in the dataframe
    latlong = row[1]['ll']  # get the latitude and longitude of each crash
    latlong_list.append(latlong)  # add latlong to dictionary


In [115]:
test = latlong_list[0:2]
test

['39.15920668,-86.52587356', '39.16144,-86.534848']

In [131]:
response_list = []

def get_venues(test):
    for latlong in test:
        response = requests.get(url+latlong.replace(',','%2C')+'&'+radius+'&'+categoryId, headers=headers)
        response_list.append(response.text)

In [132]:
get_venues(test)

In [133]:
response_list

['{"results":[],"context":{"geo_bounds":{"circle":{"center":{"latitude":54.08329772949219,"longitude":-122.4832992553711},"radius":5000}}}}',
 '{"results":[],"context":{"geo_bounds":{"circle":{"center":{"latitude":54.08329772949219,"longitude":-122.4832992553711},"radius":5000}}}}']

In [127]:
JSON(response_list)

<IPython.core.display.JSON object>

In [102]:
bars

{'results': [],
 'context': {'geo_bounds': {'circle': {'center': {'latitude': 54.08329772949219,
     'longitude': -122.4832992553711},
    'radius': 5000}}}}

In [86]:
ll_test = bar_crash_dict['latlong'][0]
ll_test

'39.15920668,-86.52587356'

In [87]:
ll_test.replace(',','%2C')

'39.15920668%2C-86.52587356'

In [83]:
len(bar_crash_dict['latlong'])

49005

In [12]:
import requests

url = "https://api.foursquare.com/v3/places/search?"

headers = {
    "Accept": "application/json",
    "Authorization": foursquare_api
}

latlong = "ll=39.15920668%2C-86.52587356"

radius = "radius=5000"

categoryId = "categories=13003"

response = requests.request("GET", url+latlong+'&'+radius+'&'+categoryId, headers=headers)

print(response.text)

{"results":[{"fsq_id":"507c2800e4b0b5e9215c9444","categories":[{"id":13003,"name":"Bar","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/nightlife/pub_","suffix":".png"}},{"id":13068,"name":"American Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":1151,"geocodes":{"main":{"latitude":39.166841,"longitude":-86.535183},"roof":{"latitude":39.166841,"longitude":-86.535183}},"link":"/v3/places/507c2800e4b0b5e9215c9444","location":{"address":"101 N College Ave","census_block":"181050001001011","country":"US","cross_street":"","dma":"Indianapolis","formatted_address":"101 N College Ave, Bloomington, IN 47404","locality":"Bloomington","neighborhood":["Arlington"],"postcode":"47404","region":"IN"},"name":"The Tap","related_places":{},"timezone":"America/Indiana/Indianapolis"},{"fsq_id":"5249fca911d20ce66163e75a","categories":[{"id":13009,"name":"Cocktail Bar","icon":{"prefix":"https://ss3.4sqi.net/img/categorie

In [15]:
from IPython.display import JSON

JSON(response.json())

<IPython.core.display.JSON object>

In [19]:
from pprint import pprint

In [20]:
pprint(response.json())

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 39.15920668,
                                                  'longitude': -86.52587356},
                                       'radius': 22000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
                                       'suffix': '.png'},
                              'id': 13003,
                              'name': 'Bar'},
                             {'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
                                       'suffix': '.png'},
                              'id': 13068,
                              'name': 'American Restaurant'}],
              'chains': [],
              'distance': 1151,
              'fsq_id': '507c2800e4b0b5e9215c9444',
              'geocodes': {'main': {'latitude': 39.166841,
                                    'longitude': -86.535183},
                           'roof'

In [21]:
import json

In [28]:
type(response.json())

dict

In [32]:
# bars = json.dumps(response.json())

In [36]:
bars = response.json()

In [35]:
pprint(response.json())

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 39.15920668,
                                                  'longitude': -86.52587356},
                                       'radius': 22000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
                                       'suffix': '.png'},
                              'id': 13003,
                              'name': 'Bar'},
                             {'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
                                       'suffix': '.png'},
                              'id': 13068,
                              'name': 'American Restaurant'}],
              'chains': [],
              'distance': 1151,
              'fsq_id': '507c2800e4b0b5e9215c9444',
              'geocodes': {'main': {'latitude': 39.166841,
                                    'longitude': -86.535183},
                           'roof'

In [50]:
bars_df = pd.json_normalize(bars, record_path ='results')
bars_df['name']

0                                          The Tap
1                                 Cardinal Spirits
2                  Upland Brewing Company Brew Pub
3                       Switchyard Brewing Company
4                                    The Back Door
5    Upland Brewing Company Brewery & Tasting Room
6                                           C3 Bar
7                            Feast Market & Cellar
8                            Big Woods Bloomington
9                  The Irish Lion Restaurant & Pub
Name: name, dtype: object

In [51]:
len(bars_df['name'])

10

In [56]:
bars_df.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.country,location.cross_street,location.dma,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.po_box,related_places.children
0,507c2800e4b0b5e9215c9444,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],1151,/v3/places/507c2800e4b0b5e9215c9444,The Tap,America/Indiana/Indianapolis,39.166841,-86.535183,39.166841,...,US,,Indianapolis,"101 N College Ave, Bloomington, IN 47404",Bloomington,[Arlington],47404,IN,NaN,NaN
1,5249fca911d20ce66163e75a,"[{'id': 13009, 'name': 'Cocktail Bar', 'icon':...",[],966,/v3/places/5249fca911d20ce66163e75a,Cardinal Spirits,America/Indiana/Indianapolis,39.156283,-86.536519,39.156283,...,US,,Indianapolis,"922 S Morton St, Bloomington, IN 47403",Bloomington,NaN,47403,IN,P.O. Box 2282,NaN
2,4ae33784f964a520cd9121e3,"[{'id': 13025, 'name': 'Wine Bar', 'icon': {'p...",[],1869,/v3/places/4ae33784f964a520cd9121e3,Upland Brewing Company Brew Pub,America/Indiana/Indianapolis,39.173503,-86.537557,39.173503,...,US,at Rogers St.,Indianapolis,"350 W 11th St (at Rogers St.), Bloomington, IN...",Bloomington,[Arlington],47404,IN,NaN,NaN
3,5b07718bd3cce8002cd23f9b,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],1393,/v3/places/5b07718bd3cce8002cd23f9b,Switchyard Brewing Company,America/Indiana/Indianapolis,39.170241,-86.533919,39.170241,...,US,9th Street,Indianapolis,"419 N Walnut St (9th Street), Bloomington, IN ...",Bloomington,[Arlington],47404,IN,NaN,"[{'fsq_id': '5f2c92aca9d86a7303abbdce', 'name'..."
4,5064d1ebe4b0675fa422bb50,"[{'id': 10032, 'name': 'Night Club', 'icon': {...",[],998,/v3/places/5064d1ebe4b0675fa422bb50,The Back Door,America/Indiana/Indianapolis,39.165239,-86.534629,NaN,...,US,,Indianapolis,"207 S College Ave, Bloomington, IN 47404",Bloomington,[Arlington],47404,IN,NaN,NaN


In [70]:
for bar in list(bars_df.iterrows()):
    print(bar[1]['name'])

The Tap
Cardinal Spirits
Upland Brewing Company Brew Pub
Switchyard Brewing Company
The Back Door
Upland Brewing Company Brewery & Tasting Room
C3 Bar
Feast Market & Cellar
Big Woods Bloomington
The Irish Lion Restaurant & Pub


In [76]:
for row in list(data.iterrows()):
    print(row[1]['ll'])

39.15920668,-86.52587356
39.16144,-86.534848
39.14978027,-86.56889006
39.165655,-86.57595635
39.164848,-86.57962482
39.12666969,-86.53136998
39.150825,-86.584899
39.19927216,-86.63702393
39.16461021,-86.57913007
39.16344009,-86.55128002
39.145264,-86.577616
39.136048,-86.518496
39.16664,-86.53070106
39.12678972,-86.58258006
39.15379018,-86.53875008
39.107344,-86.5264
39.179248,-86.547104
39.179248,-86.547104
39.17830276,-86.52934265
39.145264,-86.577616
39.09571236,-86.54595204
39.12144,-86.526496
39.16692661,-86.4572165
39.02843027,-86.40780689
39.121584,-86.489744
39.16424,-86.49312041
39.17093258,-86.509391
39.15263367,-86.49272919
39.13608,-86.538096
39.125312,-86.610496
39.17187028,-86.4167107
39.16461021,-86.57913007
39.121648,-86.539088
38.992016,-86.537248
39.168656,-86.5468
39.17900187,-86.52832631
39.171632,-86.523536
39.134144,-86.642864
39.14192595,-86.54314176
39.19115236,-86.51175974
39.14728113,-86.58089831
39.215152,-86.5928
39.21210074,-86.58736299
39.208048,-86.528704

In [75]:
type(row)

tuple

# World Weather Online API

World Weather Online API is [here](https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx)

1. Sign up for FREE api key if you haven't done that before (it's free for **30 days**).
2. For each crush, get the weather for the location and date.
3. Find a relationship between the weather and severity of the crash.

Hints:

* pull weather only for smaller sample of crashes (250 or so) due to API limits
* for sending HTTP requests check out "requests" library [here](http://docs.python-requests.org/en/master/)


In [193]:
import requests
import time
api_key = os.environ["<>"]